In [1]:
from new import *
from IPython.core.display import display
%reload_ext autoreload
%autoreload 2
pd.set_option('display.max_columns', 30)
ROOT = Path.cwd()
DATA_ROOT = ROOT.parent.parent / 'Datasets' / '2ndData'

In [13]:
def calculate_max_angle(fromX,fromY,fromZ,toX,toY,toZ,r):
    length = np.sqrt((toX-fromX)**2 + (toY-fromY)**2 + (toZ-fromZ)**2)
    angle_in_rad=np.arcsin(r/length)

    return np.rad2deg(angle_in_rad)

def calculate_current_angle(headX,headY,headZ,fromX,fromY,fromZ,toX,toY,toZ):
    import vg
    vector1 = np.array([headX,headY,headZ])
    vector2 = np.array([toX-fromX,toY-fromY,toZ-fromZ])
    return vg.angle(vector1,vector2)

def add_columns(holo):
    rs = []
    thetas = []
    phis = []
    for index, row in holo.iterrows():
        x = row['TargetPositionX'] - row['HeadPositionX']
        y = row['TargetPositionY'] - row['HeadPositionY']
        z = row['TargetPositionZ'] - row['HeadPositionZ']
        [r, theta, phi] = Analysing_functions.asSpherical([x, z, y])
        rs.append(r)
        thetas.append(90 - theta)
        phis.append(90 - phi)

    holo['R'] = rs
    holo['Theta'] = thetas
    holo['Phi'] = phis
    holo['TargetVertical'] = holo.HeadRotationX + holo.Theta
    holo['TargetHorizontal'] = holo.HeadRotationY - holo.Phi
    for col in ['HeadForwardX', 'HeadForwardY', 'HeadForwardZ']:
        holo[str(col) + '_next'] = holo[col].shift(1)
    holo['time_interval'] = holo.Timestamp.diff()
    holo['angle_speed'] = holo.apply(lambda x: angle_velocity(x.HeadForwardX, x.HeadForwardY, x.HeadForwardZ,
                                                                        x.HeadForwardX_next, x.HeadForwardY_next,
                                                                        x.HeadForwardZ_next,
                                                                        x.time_interval), axis=1)
    return holo

def angle_velocity(x1, y1, z1, x2, y2, z2, _time):
    import vg
    # if type(_head_forward2) is not dict: return None
    vector1 = np.array([x1, y1, z1])
    vector2 = np.array([x2, y2, z2])
    return vg.angle(vector1, vector2) / _time

subjects = subjects = range(201, 212)
poss = ['W','S']
envs = ['U','W']
targets=range(8)
# blocks=range(5)
blocks = [1,2,3,4]
# subjects= [201]
# poss=['W']
# envs=['W']
# targets=[0]
# blocks=[0]
output=pd.DataFrame()
# TODO : summary save for each subject
for subject in subjects:
    [imu_file_list, eye_file_list, hololens_file_list] = FileHandling.get_one_subject_files(subject, refined=True)
    for target, env, pos, block in itertools.product(targets, envs, poss, blocks):
        current_info = [target, env, pos, block]
        try:
            print('analysing',subject,current_info)

            holo = FileHandling.file_as_pandas(FileHandling.get_file_by_info(hololens_file_list, current_info))
            holo.Timestamp  = holo.Timestamp - holo.Timestamp[0]
            holo = add_columns(holo)
            if holo[holo.TargetEntered=='target_'+str(target)].shape[0]<1 :
                print("Fail in trial")
                continue
            start_index = holo[holo.TargetEntered=='target_'+str(target)].index[0]

            # Watch after first target-on


            holo_after_target_on = holo[holo.index > start_index]
            start_time = holo_after_target_on.Timestamp.values[0]
            # --- Behavioural statistics ---
            # Accuracy : mean distance
            accuracy_vertical =-holo_after_target_on.Theta.mean() - holo_after_target_on.HeadRotationX.mean()
            accuracy_horizontal = holo_after_target_on.Phi.mean() - holo_after_target_on.HeadRotationY.mean()
            # Precision : standard deviation
            precision_vertical = (holo_after_target_on.TargetVertical).std()
            precision_horizontal = (holo_after_target_on.TargetHorizontal).std()
            # print('accuracy:',accuracy_vertical,accuracy_horizontal)
            # print('precision:',precision_vertical,precision_horizontal)

            # --- Target-wise ---
            if env=='W':
                r=0.25
            else:
                r=0.25/5
            holo['max_angle_distance'] = holo.apply(lambda x:calculate_max_angle(x['HeadPositionX'],x['HeadPositionY'],x['HeadPositionZ'],
                                                                                 x['TargetPositionX'],x['TargetPositionY'],x['TargetPositionZ'],r),axis=1)
            holo['current_angle_distance'] = holo.apply(lambda x: calculate_current_angle(x['HeadForwardX'],x['HeadForwardY'],x['HeadForwardZ'],x['HeadPositionX'],x['HeadPositionY'],x['HeadPositionZ'],
                                                                                 x['TargetPositionX'],x['TargetPositionY'],x['TargetPositionZ']),axis=1)
            holo['targetOn'] = holo.apply(lambda x: x['current_angle_distance'] < x['max_angle_distance'], axis=1)
            holo['targetOn_prev'] = holo.targetOn.shift(1)
            dwells=list()
            temp_dwell=list()
            for index,row in holo.iterrows():
                if row['targetOn'] == True and row['targetOn_prev']==False:
                    temp_dwell=list()
                    temp_dwell.append(row)
                elif row['targetOn'] == True and row['targetOn_prev']==True:
                    temp_dwell.append(row)
                elif row['targetOn'] == False and row['targetOn_prev']==True:
                    if temp_dwell:
                        dwells.append(pd.DataFrame(temp_dwell))
                    temp_dwell=list()
                if index==holo.index[-1]:
                    if temp_dwell :
                        dwells.append(pd.DataFrame(temp_dwell))


            target_on_count = len(dwells)
            if target_on_count > 0:
                longest_target_on = max([x.Timestamp.values[-1]-x.Timestamp.values[0] for x in dwells])
            else:
                longest_target_on=-1
            mean_speeds=list()
            target_in_speeds=list()
            target_out_speeds=list()
            last_100ms_speeds=list()
            for dwell in dwells:
                mean_speeds.append(dwell.angle_speed.mean())
                target_in_speeds.append(dwell.angle_speed.values[0])
                target_out_speeds.append(dwell.angle_speed.values[-1])
                dwell_last100 = dwell[dwell.Timestamp > dwell.Timestamp.values[-1]-100/1000]
                last_100ms_speeds.append(dwell_last100.angle_speed.mean())
            thresholds=[100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500]
            for threshold in thresholds:
                dwell_threshold = threshold
                success_dwells=list()
                total_duration=0
                first_dwell_timestamp = -1;first_success_dwell_timestamp=-1
                prior_count=0
                for index, dwell in enumerate(dwells):
                    if index==0: first_dwell_timestamp=dwell.Timestamp.values[0]
                    duration = dwell.Timestamp.values[-1]-dwell.Timestamp.values[0]
                    total_duration += duration
                    if duration >dwell_threshold/1000:
                        if first_success_dwell_timestamp<0:
                            prior_count=index
                            first_success_dwell_timestamp=dwell.Timestamp.values[0]
                        success_dwells.append(dwell)

                success_dwell_count = len(success_dwells)
                result = dict(
                    subject=subject,target=target, env=env, pos=pos, block=block,
                    accuracy_vertical=accuracy_vertical,accuracy_horizontal=accuracy_horizontal,
                    precision_vertical=precision_vertical,precision_horizontal=precision_horizontal,
                    dwell_threshold=dwell_threshold,dwells=dwells,success_dwells=success_dwells,
                    target_on_count=target_on_count,success_dwell_count=success_dwell_count,
                    total_duration = total_duration,mean_duration = total_duration/target_on_count,
                    first_dwell_timestamp = first_dwell_timestamp, first_success_dwell_timestamp=first_success_dwell_timestamp,longest_target_on=longest_target_on,
                    mean_speeds=mean_speeds,target_in_speeds=target,target_out_speeds=target_out_speeds,last_100ms_speeds=last_100ms_speeds,
                    prior_count=prior_count
                )
                output=output.append(result,ignore_index=True)
        except ValueError as err:
            print(err, current_info)
print(output.shape)

analysing 201 [0, 'U', 'W', 1]
analysing 201 [0, 'U', 'W', 2]
analysing 201 [0, 'U', 'W', 3]
analysing 201 [0, 'U', 'W', 4]
analysing 201 [0, 'U', 'S', 1]
analysing 201 [0, 'U', 'S', 2]
analysing 201 [0, 'U', 'S', 3]
analysing 201 [0, 'U', 'S', 4]
analysing 201 [0, 'W', 'W', 1]
analysing 201 [0, 'W', 'W', 2]
analysing 201 [0, 'W', 'W', 3]
analysing 201 [0, 'W', 'W', 4]
analysing 201 [0, 'W', 'S', 1]
analysing 201 [0, 'W', 'S', 2]
analysing 201 [0, 'W', 'S', 3]
analysing 201 [0, 'W', 'S', 4]
analysing 201 [1, 'U', 'W', 1]
analysing 201 [1, 'U', 'W', 2]
analysing 201 [1, 'U', 'W', 3]
analysing 201 [1, 'U', 'W', 4]
analysing 201 [1, 'U', 'S', 1]
analysing 201 [1, 'U', 'S', 2]
analysing 201 [1, 'U', 'S', 3]
analysing 201 [1, 'U', 'S', 4]
analysing 201 [1, 'W', 'W', 1]
analysing 201 [1, 'W', 'W', 2]
analysing 201 [1, 'W', 'W', 3]
analysing 201 [1, 'W', 'W', 4]
analysing 201 [1, 'W', 'S', 1]
analysing 201 [1, 'W', 'S', 2]
analysing 201 [1, 'W', 'S', 3]
analysing 201 [1, 'W', 'S', 4]
analysin

In [14]:
output.to_pickle("summary.pkl")

In [68]:
summary = pd.read_pickle("summary.pkl")